In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()




<Logger detectron2 (DEBUG)>

In [2]:
# Librerias de uso común
import numpy as np
import torch
from scipy.spatial import distance
import os, json, cv2, random, shutil
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow

# Utilidades del detectron2

from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2 import structures
from detectron2.structures.boxes import Boxes
from detectron2 import model_zoo